In [3]:
# Author: @VK_Intel
# Find: @malwrhunterteam
# Ref: https://twitter.com/malwrhunterteam/status/1100376870133161984
# 2019-02-26: unpacked Cryptomix ransomware 
# SHA-256: 653a9e39420d1fc25b4e66941f8aae039a7133ba873b44fcf3ee58638241aca6

'''
The malware contains a digital certificate
[+] ALINA LTD
Status  Valid
Issuer Sectigo RSA Code Signing CA
Valid from 12:00 AM 02/22/2019
Valid to 11:59 PM 02/22/2020
Valid usage Code Signing
Algorithm sha256RSA
Thumbprint D92D20D8739F8DF0BDB9FDB5668274910DC05900
Serial number 4C F3 3D 7E 47 8A B7 0C 52 7D D4 5A 68 DB F3 5E
[+] Sectigo RSA Code Signing CA
Status  Valid
Issuer USERTrust RSA Certification Authority
Valid from 12:00 AM 11/02/2018
Valid to 11:59 PM 12/31/2030
Valid usage Code Signing, Timestamp Signing
Algorithm sha384RSA
Thumbprint 94C95DA1E850BD85209A4A2AF3E1FB1604F9BB66
Serial number 1D A2 48 30 6F 9B 26 18 D0 82 E0 96 7D 33 D3 6A
[+] USERTrust Secure™
Status  Valid
Issuer USERTrust RSA Certification Authority
Valid from 12:00 AM 02/01/2010
Valid to 11:59 PM 01/18/2038
Valid usage Server Auth, Client Auth, Email Protection, Code Signing, Timestamp Signing, EFS, IPSEC Tunnel, IPSEC User
Algorithm sha384RSA
Thumbprint 2B8F1B57330DBBA2D07A6C51F70EE90DDAB9AD8E
Serial number 01 FD 6D 30 FC A3 CA 51 A8 1B BC 64 0E 35 03 2D

'''


import pefile
import argparse

#the variable used to store our raw config data
first_blob = "" 
second_blob = "" 
offset = 0x0
size = 0x0

# Load key bytearray
key=bytearray(b'JLKHFVIjewhyur3ikjfldskfkl23j3iuhdnfklqhrjjio2ljkeosfjh7823763647823hrfuweg56t7r6t73824y78Clop')
# They use the same 0x42 XOR key
xor_key = 0x42

# Walk through the PE sections and get their images
def get_section_blob(peL, rsrc_name):
    for rsrc in peL.DIRECTORY_ENTRY_RESOURCE.entries:
        if rsrc.name.__str__() == rsrc_name:
            for entry in rsrc.directory.entries:
                offset = entry.directory.entries[0].data.struct.OffsetToData
                size = entry.directory.entries[0].data.struct.Size
    return peL.get_memory_mapped_image()[offset:offset+size]

# Decode the sections using a custor XOR functions
def xor_decode(key, blob):
    encoded = bytearray(blob)
    for i in range(len(encoded)):
        encoded[i] ^= key[i % xor_key]
        encoded.decode('ascii', 'replace')
    return encoded

# Process the unpacked sample
#pe = pefile.PE("PATH_TO_UNPACKED_CRYPTOMIX_RANSOMWARE")
pe = pefile.PE("/Users/vk/Downloads/2019-02-26-cryptomix-ransomware.unpacked.vk.exe")
rsrc = get_section_blob(pe,"RC_DATA")
rsrc2 = get_section_blob(pe,"RC_HTML1")
dec_rsrc1 = xor_decode(key, rsrc)
dec_rsrc2 = xor_decode(key, rsrc2)
if dec_rsrc1 and dec_rsrc2 is not None:
    print("\n[*] CryptoMix Ransomware Section Custom XOR 0x42 Decoder->\n")
    print("\n[*] First decoded resource section: \n\n%s" % (dec_rsrc1.decode("utf-8") ))
    print("\n[*] Second decoded resource section: \n\n%s" % (dec_rsrc2.decode("utf-8") ))


[*] CryptoMix Ransomware Section Custom XOR Decoder ->


[*] First decoded resource section: 

*_*Your networks has been penetrated*_*
All files on each host in the networks have been encrypted with a strong algorithm!
Backups were either encrypted or deleted or backup disks were formatted!
Shadow copies also removed, so F-8 or any other methods may damage encrypted data but not recover!
We exclusively have decryption software for your situation!
===No DECRYPTION software is AVAILABLE in the PUBLIC===
- DO NOT DELETE readme files.
- DO NOT RENAME OR MOVE the encrypted and readme files.
- DO NOT RESET OR SHUTDOWN – files may be damaged.
---THIS MAY LEAD TO THE IMPOSSIBILITY OF RECOVERY OF THE CERTAIN FILES---
---ALL REPAIR TOOLS ARE USELESS AND CAN DESTROY YOUR FILES IRREVERSIBLY---
If you want to restore your files write to email!
[CONTACTS ARE AT THE BOTTOM OF THE SHEET] and attach 2 - 6 encrypted files!!!
[Less than 7 Mb each, non-archived and your files should not contain valuable 